## Machine Learning Pipeline - Data Analysis
In the following notebooks, we will go through the implementation of each steps in the Machine Learning Pipeline: 

1. Data Analysis
2. **Feature Engineering**
3. Feature Selection
4. Model Training
5. Obtaining Predictions/Scoring

### Rossman Store Sales Prediction
The aim of the project is to build an end-to-end machine learning model to predict the sales of a given store and a set of inputs, including the promotions, competition, school and state holidays, seasonality, and locality.

In [1]:
import math

# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# for the yeo-johnson transformation
import scipy.stats as stats

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to save the trained scaler class
import joblib

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

In [2]:
# load dataset
df_sales = pd.read_csv('train.csv')
df_store = pd.read_csv('store.csv')

# rows and columns of the data
print(df_sales.shape)
print(df_store.shape)

(914629, 9)
(1115, 10)


C:\Users\benja\anaconda3\envs\test-ml-gcash-rossmann-research\lib\site-packages\IPython\core\interactiveshell.py:3012: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Merge
df_raw = pd.merge( df_sales, df_store, how = 'left', on = 'Store' )

print(df_raw.info())
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 914629 entries, 0 to 914628
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Store                      914629 non-null  int64  
 1   DayOfWeek                  914629 non-null  int64  
 2   Date                       914629 non-null  object 
 3   Sales                      914629 non-null  int64  
 4   Customers                  914629 non-null  int64  
 5   Open                       914629 non-null  int64  
 6   Promo                      914629 non-null  int64  
 7   StateHoliday               914629 non-null  object 
 8   SchoolHoliday              914629 non-null  int64  
 9   StoreType                  914629 non-null  object 
 10  Assortment                 914629 non-null  object 
 11  CompetitionDistance        912263 non-null  float64
 12  CompetitionOpenSinceMonth  623849 non-null  float64
 13  CompetitionOpenSinceYear   62

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,4,2015-04-30,6228,650,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,4,2015-04-30,6884,716,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,4,2015-04-30,9971,979,1,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,4,2015-04-30,16106,1854,1,1,0,0,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,4,2015-04-30,6598,729,1,1,0,0,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [4]:
# copy dataset
df1 = df_raw.copy()

In [5]:
# drop all rows with zero Sales amount
df1 = df1[df1['Sales'] > 0]

print(df1.shape)
df1.head()

(759848, 18)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,4,2015-04-30,6228,650,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,4,2015-04-30,6884,716,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,4,2015-04-30,9971,979,1,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,4,2015-04-30,16106,1854,1,1,0,0,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,4,2015-04-30,6598,729,1,1,0,0,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


### Separate dataset into train and test
When we engineer features, some techniques learn parameters from data. It is important to learn these parameters only from the train set. This is to avoid over-fitting.

Our feature engineering techniques will learn:

* mean
* mode
* exponents from the yeo-johnson
* category frequency
* and category to number mappings

from the train set.

In [6]:
# separate into train and test set

X_train, X_test, y_train, y_test = train_test_split(
    df1.drop(['Store', 'Sales'], axis=1), # predictive variables
    df1['Sales'], # target
    test_size=0.2, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)

X_train.shape, X_test.shape

((607878, 16), (151970, 16))

### Feature Engineering
In the following cells, we will engineer the variables so that we tackle:
1. Missing values
2. Temporal variables
3. Non-Gaussian distributed variables
4. Categorical variables: convert strings to numbers
5. Put the variables in a similar scale

In [7]:
# for target, we apply the logarithm (log1p)
y_train = np.log1p(y_train) # = np.log(y_train + 1)
y_test = np.log1p(y_test)

In [8]:
#  transform datatype of the variable date to datetime
X_train['Date'] = pd.to_datetime(X_train['Date'])
X_test['Date'] = pd.to_datetime(X_test['Date'])

### Missing Values
#### Categorical variables
We will replace missing values with the string "missing" in those variables with at least 50% of missing data. <br>
Alternatively, we will replace missing data with the most frequent category in those variables that contain fewer observations without values.

In [9]:
# identify the categorical variables; capture all those with type "object"
cat_vars = [var for var in df1.columns if df1[var].dtype == 'O' and var != 'Date']
cat_vars

['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']

In [10]:
# cast all variables as categorical
X_train[cat_vars] = X_train[cat_vars].astype('O')
X_test[cat_vars] = X_test[cat_vars].astype('O')

In [11]:
# make a list of the categorical variables that contain missing values

cat_vars_with_na = [
    var for var in cat_vars
    if X_train[var].isnull().sum() > 0
]

# print percentage of missing values per variable
X_train[cat_vars_with_na ].isnull().mean().sort_values(ascending=False)

PromoInterval    0.502254
dtype: float64

In [12]:
# variables to impute with the string missing
with_string_missing = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean() >= 0.50]

with_string_missing    

['PromoInterval']

In [13]:
# replace missing values with new label: "Missing"

X_train[with_string_missing] = X_train[with_string_missing].fillna('Missing')
X_test[with_string_missing] = X_test[with_string_missing].fillna('Missing')

In [14]:
# check that we have no missing information in the engineered variables

X_train[cat_vars_with_na].isnull().sum()

PromoInterval    0
dtype: int64

In [15]:
# check that test set does not contain null values in the engineered variables

[var for var in cat_vars_with_na if X_test[var].isnull().sum() > 0]

[]

#### Numerical variables
To engineer missing values in numerical variables, we will:
* add a binary missing indicator variable
* and then replace the missing values in the original variable with the mean or mode

In [16]:
# identify the numerical variables

num_vars = [
    var for var in X_train.columns if var not in cat_vars and var != 'Sales'
]

num_vars

['DayOfWeek',
 'Date',
 'Customers',
 'Open',
 'Promo',
 'SchoolHoliday',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear']

In [17]:
# make a list with the numerical variables that contain missing values
vars_with_na = [
    var for var in num_vars
    if X_train[var].isnull().sum() > 0
]

# print percentage of missing values per variable
X_train[vars_with_na].isnull().mean().sort_values(ascending=False)

Promo2SinceWeek              0.502254
Promo2SinceYear              0.502254
CompetitionOpenSinceMonth    0.317962
CompetitionOpenSinceYear     0.317962
CompetitionDistance          0.002596
dtype: float64

In [18]:
X_train['CompetitionDistance'].mode()[0]

250.0

In [19]:

for var in vars_with_na:

    if var == 'CompetitionDistance':
        # calculate the mean using the train set
        mean_val = X_train[var].mean()

        # add binary missing indicator (in train and test)
        X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
        X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)

        # replace missing values by a unique value (in train and test)
        X_train[var].fillna('999999', inplace=True)
        X_test[var].fillna('999999', inplace=True)

        # replace the unique value by the mode
        X_train[var].replace(to_replace='999999',value=mean_val,inplace=True)
        X_test[var].replace(to_replace='999999',value=mean_val,inplace=True)

    else:
        # calculate the mean using the train set
        mode_val = X_train[var].mode()[0]

        # add binary missing indicator (in train and test)
        X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
        X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)

        # replace missing values by a unique value (in train and test)
        X_train[var].fillna('999999', inplace=True)
        X_test[var].fillna('999999', inplace=True)

        # replace the unique value by the mode
        X_train[var].replace(to_replace='999999',value=mode_val,inplace=True)
        X_test[var].replace(to_replace='999999',value=mode_val,inplace=True)

# check that we have no more missing values in the engineered variables
X_train[vars_with_na].isnull().sum()    

CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2SinceWeek              0
Promo2SinceYear              0
dtype: int64

In [20]:
# check that test set does not contain null values in the engineered variables

[var for var in vars_with_na if X_test[var].isnull().sum() > 0]

[]

In [21]:
# check the binary missing indicator variables

X_train[['CompetitionDistance_na', 'Promo2SinceWeek_na', 'CompetitionOpenSinceYear_na']].head()

,CompetitionDistance_na,Promo2SinceWeek_na,CompetitionOpenSinceYear_na
909242,0,0,1
865171,0,0,1
826279,0,0,0
524907,0,0,1
589075,0,1,1


### Temporal variables

In [22]:
# year
X_train['year'] = X_train['Date'].dt.year
X_test['year'] = X_test['Date'].dt.year



X_train.head()

,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionDistance_na,CompetitionOpenSinceMonth_na,CompetitionOpenSinceYear_na,Promo2SinceWeek_na,Promo2SinceYear_na,year
909242,6,2013-01-05,442,1,0,0,0,d,a,850.0,9.0,2013.0,1,18.0,2011.0,"Feb,May,Aug,Nov",0,1,1,0,0,2013
865171,4,2013-02-14,479,1,0,0,0,d,c,310.0,9.0,2013.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct",0,1,1,0,0,2013
826279,4,2013-03-21,693,1,1,0,0,d,a,1120.0,5.0,2007.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",0,0,0,0,0,2013
524907,1,2013-12-16,1617,1,1,0,0,a,c,20620.0,9.0,2013.0,1,37.0,2009.0,"Jan,Apr,Jul,Oct",0,1,1,0,0,2013
589075,6,2013-10-19,438,1,0,0,0,d,a,2300.0,9.0,2013.0,0,14.0,2011.0,Missing,0,1,1,1,1,2013


In [23]:
# we will capture the time elapsed (using the year variable)
def elapsed_years(df, var):
    # capture difference between the year variable
    # and the year in the nearest competitor was opened
    # and when the store started participating in Promo2
    df[var] = abs(df['year'] - df[var])
    return df

In [24]:
for var in ['CompetitionOpenSinceYear', 'Promo2SinceYear']:
    X_train = elapsed_years(X_train, var)
    X_test = elapsed_years(X_test, var)

In [25]:
# drop columns
X_train.drop(['Date',
              'Promo2SinceWeek',
              'year',
              'Promo2SinceWeek_na'], axis=1, inplace=True)

X_test.drop(['Date',
             'Promo2SinceWeek',
             'year',
             'Promo2SinceWeek_na'], axis=1, inplace=True)

### Numerical variable transformation
#### Yeo-Johnson transformation
We will apply the Yeo-Johnson transformation to CompetitionDistance and Customers


In [ ]:
# ["Customers", "CompetitionDistance"]